In [1]:
from requests import get
import bs4
from bs4 import BeautifulSoup


import pandas as pd
import numpy as np

from time import sleep
from random import randint

from time import time
from warnings import warn

from datetime import datetime
from datetime import timedelta
from pytz import timezone

from IPython.display import clear_output


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import re
import os
import sys

import glob
import shutil
from pathlib import Path

In [2]:
# Path of the data folder
folder_data = r'C:\Users\motodangiuuuu\Crawler'

# Settings for web driver and Proxy
chrome_driver_path = os.path.join(os.getcwd(),'chromedriver.exe')
# PROXY = "socks5://localhost:9050" # IP:PORT or HOST:PORT

# create a new Chrome session
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')
# options.add_argument('--proxy-server=%s' % PROXY)
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)
driver.implicitly_wait(0)


In [3]:
url = 'https://careerbuilder.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh-kenh-thuong-mai-dien-tu.35B5EE16.html'
driver.get(url)
sleep(3)

In [32]:
dict_detail = {}

In [33]:
element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template01 "]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup_level1=BeautifulSoup(html_of_interest, 'lxml')

In [34]:
map_div = soup_level1.select('div[class="head-left"] > div[class="title"] > h2')
dict_detail['Tên việc làm'] = map_div[0].text

In [35]:
dict_detail['Link'] = url

In [36]:
welfares = soup_level1.select('div[class="detail-row box-welfares"]')[0].select('li')
phuc_loi=[]
for i in welfares:
    phuc_loi.append(i.text.strip())
    dict_detail['Phúc lợi'] = phuc_loi
dict_detail['Phúc lợi']

['Chế độ bảo hiểm',
 'Phụ cấp',
 'Chế độ thưởng',
 'Đào tạo',
 'Tăng lương',
 'Công tác phí',
 'Nghỉ phép năm']

In [37]:
map_div = soup_level1.select('div[class="box-right"]')[0].select('td')
i=0
while i < len(map_div):
    dict_detail[map_div[i].text.strip()] = map_div[i+1].text.strip()
    i += 2

In [37]:
map_div = soup_level1.select('div[class="full-content"] > div')

for item in range(0, len(map_div)):
    attribute = map_div[item]
    if map_div[item].select('h3')[0].text.strip().capitalize() == "Job tags / skills:":
        jobtags_skills=[]
        tag_skills = map_div[item].select('div[class="job-tags detail-row"] > ul > li')
        if tag_skills:
            for tag in tag_skills:
                jobtags_skills.append(tag.text)
        dict_detail[map_div[item].select('h3')[0].text.strip().capitalize()] = jobtags_skills        
    else:
        att =""
        for item in attribute.select('p, ul'):
            att = att + str(item) +("\n")
            att = att.replace("<", "[").replace(">", "]")
        dict_detail[attribute.select('h3')[0].text.strip().capitalize()] = att

In [71]:
# element = driver.find_element_by_tag_name("td")
# element.click()

In [38]:
link_company_info = soup_level1.select('a[class="company"]')[0]['href']
# link_company_info
driver.get(link_company_info)
sleep(3)

In [39]:
element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="company-info"]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup_level1=BeautifulSoup(html_of_interest, 'lxml')  

In [40]:
company_info = soup_level1.select('div[class="content"]')[0]

In [41]:
name = company_info.select('p')[0].text
address = company_info.select('p')[1].text
dict_detail['Tên công ty'] = name
dict_detail['Địa chỉ'] = address

In [42]:
company_detail = company_info.select('ul > li')
company_detail
for i in range(len(company_detail)):
    role = company_detail[i].text.split(":")[0].strip()
    detail = company_detail[i].text.split(":")[1].strip()
    dict_detail[role] = detail

In [43]:
df = pd.DataFrame.from_dict(dict_detail, orient='index')
df = df.transpose()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Tên việc làm         1 non-null      object
 1   Link                 1 non-null      object
 2   Phúc lợi             1 non-null      object
 3   Ngành nghề           1 non-null      object
 4   Lương                1 non-null      object
 5   Hình thức            1 non-null      object
 6   Cấp bậc              1 non-null      object
 7   Kinh nghiệm          1 non-null      object
 8   Hết hạn nộp          1 non-null      object
 9   Ngày cập nhật        1 non-null      object
 10  Địa điểm             1 non-null      object
 11  Mô tả công việc      1 non-null      object
 12  Yêu cầu công việc    1 non-null      object
 13  Thông tin khác       1 non-null      object
 14  Job tags / skills:   1 non-null      object
 15  Tên công ty          1 non-null      object
 16  Địa chỉ     

In [44]:
df

Tên việc làm  \
0  Nhân viên kinh doanh kênh thương mại điện tử   

                                                Link  \
0  https://careerbuilder.vn/vi/tim-viec-lam/nhan-...   

                                            Phúc lợi  \
0  [Chế độ bảo hiểm, Phụ cấp, Chế độ thưởng, Đào ...   

                                          Ngành nghề  \
0  Bán hàng / Kinh doanh, Dịch vụ khách hàng, Thự...   

                        Lương             Hình thức    Cấp bậc Kinh nghiệm  \
0  8,000,000 - 10,000,000 VND  Nhân viên chính thức  Nhân viên   1 - 2 Năm   

  Hết hạn nộp Ngày cập nhật     Địa điểm  \
0  15/04/2021    23/03/2021  Hồ Chí Minh   

                                     Mô tả công việc  \
0  [p]1 - Hỗ trợ và tư vấn bán hàng tại cửa hàng[...   

                                   Yêu cầu công việc  \
0  [p]+ Có kinh nghiệm tối thiểu từ 1 - 2 năm về ...   

                                      Thông tin khác  \
0  [ul]\n[li] Bằng cấp:\n                Trung cấ...   

                                  Job tags / skills:  \
0  [ Đại diện kinh doanh,  Nhân viên phòng kinh d...   

                                    Tên công ty  \
0  SHIN Cà Phê - Công ty CP cà phê Golden Beans   

                               Địa chỉ Qui mô công ty Loại hình hoạt động  
0  13 Nguyễn Thiệp, P.Bến Nghé, Quận 1          30-50             Cổ phần